In [1]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2020-12-13 08:41:34--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  12.0MB/s    in 4.2s    

2020-12-13 08:41:38 (10.3 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [2]:
!python3 -m nltk.downloader stopwords
!python3 -m nltk.downloader universal_tagset
!python3 -m spacy download en # download the english model

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-huedm1re
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-huedm1re
     |████████████████████████████████| 245kB 4.2MB/s 
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8763598 sha256=d2dc600a7699c896faf64736e00524c430f0a9c5c0fa5468c2c5efefef96c2ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-2prt80pd/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke


In [4]:
!pip install flair

     |████████████████████████████████| 450kB 4.3MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 983kB 41.0MB/s 
     |████████████████████████████████| 19.7MB 1.2MB/s 
     |████████████████████████████████| 798kB 43.2MB/s 
     |████████████████████████████████| 1.1MB 42.9MB/s 
     |████████████████████████████████| 1.3MB 37.5MB/s 
     |████████████████████████████████| 890kB 41.2MB/s 
     |████████████████████████████████| 2.9MB 39.0MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=7891ee6e92f42f8fd3d3cde88fe153d465fe04471ae6980cdad932faf6839ecc
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=747add9fabc63726f21d7ac3481bfae5b5dc1a63d60e45024fdd0672ab4d6939
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d66

In [52]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, TransformerWordEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        WordEmbeddings('glove'),
                                        FlairEmbeddings('news-forward-fast'),
                                        FlairEmbeddings('news-backward-fast')
                                       ])

In [27]:
with open('some.txt', 'r') as f:
    s = f.read()

In [48]:
we = {}

In [49]:
from flair.data import Sentence
sentence = Sentence(s)

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)
# now check out the embedded tokens.
for token in sentence:
    we[token.text] = token.embedding


In [33]:
import pke

# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN', 'ADJ'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

# 1. create a PositionRank extractor.
extractor = pke.unsupervised.PositionRank()

# 2. load the content of the document.
extractor.load_document(input='some.txt',
                        language='en',
                        normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=3)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=20)

In [34]:
keyphrases

[('iphone car charger', 0.14101853191889602),
 ('lower quality charger', 0.10601647037245157),
 ('great car charger', 0.10240909301062759),
 ('at&t car charger', 0.09949252343014053),
 ('car charger', 0.07808359516278174),
 ('new iphone', 0.07399924993845962),
 ('great charger', 0.07276030966093779),
 ('charges', 0.070063824218452),
 ('girlfriends iphone', 0.06749793515301374),
 ('iphone', 0.06293493675611427),
 ('fast rate', 0.05505831203358287),
 ('second usb port', 0.054341103254313364),
 ('usb port', 0.0522508237857858),
 ('days usb chargers', 0.05195780788990237),
 ('broken defective charger', 0.05036840613697882),
 ('charger', 0.04843481181309195),
 ('usb cable', 0.042639218490218664),
 ('usb slot', 0.03987977315211335),
 ('usb', 0.03657340724063278),
 ('great price', 0.03614718800498726)]

In [35]:
import torch
def get_mean_emb(phrase):
    ws = phrase.split(" ")
    vect = torch.zeros(list(we.values())[0].numpy().shape)
    c = 1
    for i in ws:
        try:
            vect += we[i]
            c += 1
        except KeyError:
            pass
    return (vect/c).numpy()

In [50]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([   get_mean_emb(i) for i,j in keyphrases   ])

kmeans = KMeans(n_clusters=8, random_state=0).fit(X)
kmeans.labels_
# kmeans.cluster_centers_

array([0, 1, 0, 0, 0, 1, 0, 7, 2, 5, 1, 6, 6, 4, 0, 0, 3, 3, 5, 1],
      dtype=int32)

In [44]:
from sklearn.cluster import OPTICS
import numpy as np
X = np.array([   get_mean_emb(i) for i,j in keyphrases   ])
clustering = OPTICS(min_samples=2).fit(X)
clustering.labels_

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/_optics.py:802: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


array([ 0, -1,  1,  0,  0, -1,  1, -1,  2,  2, -1,  3,  3, -1, -1, -1, -1,
       -1,  2, -1])

In [45]:
p = np.array(keyphrases)

In [51]:
for i in range(-1, 10):
    print(p[kmeans.labels_ == i])
    # print(p[clustering.labels_ == i])
    print("  ")

[]
  
[['iphone car charger' '0.14101853191889602']
 ['great car charger' '0.10240909301062759']
 ['at&t car charger' '0.09949252343014053']
 ['car charger' '0.07808359516278174']
 ['great charger' '0.07276030966093779']
 ['broken defective charger' '0.05036840613697882']
 ['charger' '0.04843481181309195']]
  
[['lower quality charger' '0.10601647037245157']
 ['new iphone' '0.07399924993845962']
 ['fast rate' '0.05505831203358287']
 ['great price' '0.03614718800498726']]
  
[['girlfriends iphone' '0.06749793515301374']]
  
[['usb cable' '0.042639218490218664']
 ['usb slot' '0.03987977315211335']]
  
[['days usb chargers' '0.05195780788990237']]
  
[['iphone' '0.06293493675611427']
 ['usb' '0.03657340724063278']]
  
[['second usb port' '0.054341103254313364']
 ['usb port' '0.0522508237857858']]
  
[['charges' '0.070063824218452']]
  
[]
  
[]
  


In [ ]:
"Over the years, my household has tried multiple Bluetooth solutions for our hands-free communications needs. We\'ve had acceptable results from a solution integrated into GPS...but much of our regular driving doesn\'t include the need to load up a full-size GPS. We\'ve".replace("\'", "'")

"Over the years, my household has tried multiple Bluetooth solutions for our hands-free communications needs. We've had acceptable results from a solution integrated into GPS...but much of our regular driving doesn't include the need to load up a full-size GPS. We've"

In [24]:
myString = ". ".join(df[df['asin'] == df['asin'].unique()[1223]]['reviewText'])
with open('some.txt','w') as f:
    f.write(myString)
myString

"Charges the iPhone 4S or lower at a fast rate. Added plus is that it has a USB port on the top so you can plug another wire into it.. Now days USB chargers are built into cars. However I find myself driving a classic (paid for car and too cheap to upgrade). So I found this animal in the AT&T; store for an exorbitant price. I just purchased a new phone and was in a purchase craze and forgot to check here first.After I bought it I realized it had a light to help find and know when this critter is working.  And the USB was hidden under a slide thing. Looks like you can charge two phones at one time although I have not tried.Any way this charger has proven useful a couple of times and when not in use gives a feel of security by just knowing it is in the glove compartment.. Was in a store about to buy a car charger for my new iPhone.  (I actually still miss my blackberry!)  A lady told me I should go to Amazon to get it as I would save money.  So, I dropped the $29.95 car charger and came 

'B007XPPA28'

In [6]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

In [ ]:
from flair.data import Sentence
sentence = Sentence("i am using multiple words like this here you can see multiple two times and like also two times")

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    we[token.text] = token.embedding
    print(token)
    print(token.embedding)

Token: 1 i
tensor([-4.6539e-02,  6.1966e-01,  5.6647e-01,  ...,  7.0599e-09,
         3.4968e-08, -6.7191e-03])
Token: 2 am
tensor([-1.4022e-01,  2.4659e-02,  1.5813e-01,  ...,  1.1888e-09,
         2.6776e-09, -1.9904e-01])
Token: 3 using
tensor([-4.6851e-01,  2.6150e-01, -3.2521e-01,  ...,  1.0679e-08,
        -3.5290e-05, -1.0316e-01])
Token: 4 multiple
tensor([-1.6573e-01,  5.4371e-01, -6.2806e-01,  ...,  2.1727e-08,
         6.4301e-09, -1.5612e-02])
Token: 5 words
tensor([ 7.2607e-02,  7.0889e-01,  4.2583e-01,  ...,  6.9773e-09,
        -6.8683e-09, -8.2192e-02])
Token: 6 like
tensor([-2.6870e-01,  8.1708e-01,  6.9896e-01,  ...,  8.3529e-10,
        -8.2322e-08, -5.9224e-01])
Token: 7 this
tensor([-5.7058e-01,  4.4183e-01,  7.0102e-01,  ...,  3.2350e-08,
        -6.1616e-08, -3.5837e-01])
Token: 8 here
tensor([-4.3061e-01,  2.8297e-01,  8.0489e-01,  ...,  2.8255e-08,
         1.8531e-05, -2.0962e-01])
Token: 9 you
tensor([-4.9886e-01,  7.6602e-01,  8.9751e-01,  ...,  5.7144e-10,
